In [100]:
#imports
from tqdm import tqdm

import numpy as np
import pandas as pd

import yfinance as yf

import matplotlib.pyplot as plt
%matplotlib inline

import matplotlib
matplotlib.use("pgf")
matplotlib.rcParams.update({
    "pgf.texsystem": "pdflatex",
    'font.family': 'serif',
    'text.usetex': True,
    'pgf.rcfonts': False,
})

import plotly.express as px
from plotly.offline import init_notebook_mode
init_notebook_mode(connected=True)

In [101]:
#define constants
risk_free_rate = 0.03
trial_n = 10000

tickers = ['NKE', 'MSFT', 'DIS', 'COST']

start = '2012-1-1'
end = '2022-1-1'

In [102]:
#download data
df_close = yf.download(tickers, start, end, interval='1mo')['Close']
# output [*********************100%***********************]  4 of 4 completed
df_close.to_csv('./data/df_close.csv')

[*********************100%***********************]  4 of 4 completed


In [103]:
df_close.shape
# output (120, 4)

(120, 4)

In [104]:
print(df_close.head().to_latex())
df_close.head()

\begin{tabular}{lrrrr}
\toprule
{} &       COST &        DIS &       MSFT &        NKE \\
Date                      &            &            &            &            \\
\midrule
2012-01-01 00:00:00-05:00 &  82.269997 &  38.900002 &  29.530001 &  25.997499 \\
2012-02-01 00:00:00-05:00 &  86.059998 &  41.990002 &  31.740000 &  26.980000 \\
2012-03-01 00:00:00-05:00 &  90.800003 &  43.779999 &  32.259998 &  27.110001 \\
2012-04-01 00:00:00-04:00 &  88.180000 &  43.110001 &  32.020000 &  27.967501 \\
2012-05-01 00:00:00-04:00 &  86.389999 &  45.709999 &  29.190001 &  27.045000 \\
\bottomrule
\end{tabular}



/tmp/ipykernel_6243/79280297.py:1: FutureWarning:

In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.



,COST,DIS,MSFT,NKE
Date,,,,
2012-01-01 00:00:00-05:00,82.269997,38.900002,29.530001,25.997499
2012-02-01 00:00:00-05:00,86.059998,41.990002,31.740000,26.980000
2012-03-01 00:00:00-05:00,90.800003,43.779999,32.259998,27.110001
2012-04-01 00:00:00-04:00,88.180000,43.110001,32.020000,27.967501
2012-05-01 00:00:00-04:00,86.389999,45.709999,29.190001,27.045000


In [105]:
print(df_close.tail().to_latex())
df_close.tail()

\begin{tabular}{lrrrr}
\toprule
{} &        COST &         DIS &        MSFT &         NKE \\
Date                      &             &             &             &             \\
\midrule
2021-08-01 00:00:00-04:00 &  455.489990 &  181.300003 &  301.880005 &  164.740005 \\
2021-09-01 00:00:00-04:00 &  449.350006 &  169.169998 &  281.920013 &  145.229996 \\
2021-10-01 00:00:00-04:00 &  491.540009 &  169.070007 &  331.619995 &  167.289993 \\
2021-11-01 00:00:00-04:00 &  539.380005 &  144.899994 &  330.589996 &  169.240005 \\
2021-12-01 00:00:00-05:00 &  567.700012 &  154.889999 &  336.320007 &  166.669998 \\
\bottomrule
\end{tabular}



/tmp/ipykernel_6243/2242973060.py:1: FutureWarning:

In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.



,COST,DIS,MSFT,NKE
Date,,,,
2021-08-01 00:00:00-04:00,455.489990,181.300003,301.880005,164.740005
2021-09-01 00:00:00-04:00,449.350006,169.169998,281.920013,145.229996
2021-10-01 00:00:00-04:00,491.540009,169.070007,331.619995,167.289993
2021-11-01 00:00:00-04:00,539.380005,144.899994,330.589996,169.240005
2021-12-01 00:00:00-05:00,567.700012,154.889999,336.320007,166.669998


In [106]:
df_close.isnull().sum()

COST    0
DIS     0
MSFT    0
NKE     0
dtype: int64

In [107]:
if df_close.isnull().values.any():
    df_close.fillna(method='ffill', inplace = True) #front fills
    df_close.fillna(method='bfill', inplace = True) #back fills

print(df_close.isnull().sum().to_latex())
df_close.isnull().sum()

\begin{tabular}{lr}
\toprule
{} &  0 \\
\midrule
COST &  0 \\
DIS  &  0 \\
MSFT &  0 \\
NKE  &  0 \\
\bottomrule
\end{tabular}



/tmp/ipykernel_6243/800395844.py:5: FutureWarning:

In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.



COST    0
DIS     0
MSFT    0
NKE     0
dtype: int64

In [108]:
def plot_line(df, xlabel, ylabel, title):
    # matplotlib
    #ax = df.plot(title = title, figsize=(16, 8), ax = None)
    #actual ^^^
    ax = df.plot(title=title, figsize=(5.39749, 2.69874), ax=None)
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    ax.axhline(y=0, color='black')
    ax.legend(loc='upper left')
    plt.savefig('./plots/'+title+'.pgf')
    plt.show()

    # plotly
    fig = px.line(df, title=title)
    fig.update_layout(xaxis_title=xlabel, yaxis_title=ylabel,
                      showlegend=True, template='seaborn')
    fig.show()


In [109]:
plot_line(df_close, 'Date', 'Stock Price', 'Close Price')

In [110]:
df_pct_change = df_close.pct_change().iloc[1:] #get rid of last row
df_pct_change.head()

,COST,DIS,MSFT,NKE
Date,,,,
2012-02-01 00:00:00-05:00,0.046068,0.079434,0.074839,0.037792
2012-03-01 00:00:00-05:00,0.055078,0.042629,0.016383,0.004818
2012-04-01 00:00:00-04:00,-0.028855,-0.015304,-0.007439,0.031630
2012-05-01 00:00:00-04:00,-0.020299,0.060311,-0.088382,-0.032985
2012-06-01 00:00:00-04:00,0.099664,0.061037,0.047962,-0.188575


In [111]:
plot_line(df_pct_change, 'Date', 'Percent', 'Percent Change (monthly return)')

In [112]:
mean_return = df_pct_change.mean() * 12
mean_return

COST    0.210163
DIS     0.166885
MSFT    0.267666
NKE     0.212343
dtype: float64

In [113]:
var_risk = df_pct_change.var() * 12
var_risk

COST    0.027995
DIS     0.055262
MSFT    0.041114
NKE     0.047685
dtype: float64

In [114]:
cov_mat = df_pct_change.cov() * 12
cov_mat

,COST,DIS,MSFT,NKE
COST,0.027995,0.010468,0.010946,0.011377
DIS,0.010468,0.055262,0.016682,0.019448
MSFT,0.010946,0.016682,0.041114,0.012376
NKE,0.011377,0.019448,0.012376,0.047685


In [115]:
corr_mat = df_pct_change.corr()
corr_mat

,COST,DIS,MSFT,NKE
COST,1.000000,0.266133,0.322651,0.311372
DIS,0.266133,1.000000,0.349985,0.378852
MSFT,0.322651,0.349985,1.000000,0.279516
NKE,0.311372,0.378852,0.279516,1.000000


In [116]:
fig = px.imshow(corr_mat, title='Correlation Heatmap')
fig.show()

In [117]:
weight = 1/len(tickers)
eq_weights = [weight for i in tickers]
eq_weights

[0.25, 0.25, 0.25, 0.25]

In [118]:
eq_return = eq_weights @ (mean_return)
eq_return

0.21426419041415923

In [119]:
eq_risk = ((eq_weights @ cov_mat) @ eq_weights)**(1/2)
eq_risk

0.14462260526809836

In [120]:
eq_sharpe = (eq_return - risk_free_rate)/eq_risk
eq_sharpe

1.2741036580870198

In [121]:
efficient_frontier_data = []
for trial in tqdm(range(trial_n)):
    t_weight = np.random.random(len(tickers, ))
    t_weights = t_weight/sum(t_weight)
    t_return = t_weights @ (mean_return)
    t_risk = ((t_weights @ cov_mat) @ t_weights)**(1/2)
    t_sharpe = (t_return - risk_free_rate)/t_risk
    efficient_frontier_data.append([t_weights, t_risk, t_return, t_sharpe])

efficient_frontier_data = pd.DataFrame(efficient_frontier_data, columns=['Weights', 'Risk', 'Return', 'Sharpe'])
efficient_frontier_data.head()

100%|██████████| 10000/10000 [00:01<00:00, 8884.62it/s]


,Weights,Risk,Return,Sharpe
0,"[0.36431500901936154, 0.20711844590486436, 0.0...",0.152048,0.202686,1.135734
1,"[0.11881854708936271, 0.2132958413304896, 0.32...",0.151994,0.220172,1.251180
2,"[0.06836031014256953, 0.390308800971074, 0.219...",0.162343,0.206571,1.087646
3,"[0.027916168983564367, 0.4343766672618365, 0.4...",0.168898,0.216830,1.106168
4,"[0.312916271452558, 0.277204696538098, 0.24213...",0.143047,0.212455,1.275492


In [122]:
i_max_sharpe = efficient_frontier_data['Sharpe'].idxmax()
max_sharpe = efficient_frontier_data.iloc[i_max_sharpe, :]
max_sharpe

Weights    [0.38037441853954856, 0.008200617465075324, 0....
Risk                                                0.142669
Return                                               0.23389
Sharpe                                              1.429111
Name: 8264, dtype: object

In [123]:
i_min_risk = efficient_frontier_data['Risk'].idxmin()
min_risk = efficient_frontier_data.iloc[i_min_risk, :]
min_risk

Weights    [0.46755249951663147, 0.13165328539791524, 0.2...
Risk                                                0.137698
Return                                              0.218024
Sharpe                                              1.365481
Name: 5351, dtype: object

In [124]:
i_max_return = efficient_frontier_data['Return'].idxmax()
max_return = efficient_frontier_data.iloc[i_max_return, :]
max_return

Weights    [0.024379932967090803, 0.0594530286894517, 0.9...
Risk                                                0.191045
Return                                              0.259672
Sharpe                                              1.202188
Name: 4102, dtype: object

In [125]:
print(eq_risk, eq_return)
print(max_sharpe['Risk'], max_sharpe['Return'])


0.14462260526809836 0.21426419041415923
0.1426691754699157 0.23389010589899914


In [126]:
from plotly import graph_objects as go


def addMarker(fig, x, y, color):
    fig.add_trace(
        go.Scatter(
            x=[x],
            y=[y],
            mode="markers",
            marker_symbol='star',
            marker_size=10,
            marker_color=color
        )
    )


In [127]:
fig = px.scatter(efficient_frontier_data, x='Risk', y='Return', title='Efficient Frontier', color='Risk')
addMarker(fig, max_sharpe['Risk'], max_sharpe['Return'], 'Green')
addMarker(fig, min_risk['Risk'], min_risk['Return'], 'Cyan')
addMarker(fig, max_return['Risk'], max_return['Return'], 'Red')
fig.show()

## Process data for recent past year for same stocks and the spy

In [128]:
tickers = tickers

start = '2022-1-1'
end = '2023-1-1'

df_recent_close = yf.download(tickers, start, end, interval='1d')['Close']
df_recent_close.to_csv('./data/df_recent_close.csv')

df_spy_close = yf.download('SPY', start, end, interval='1d')['Close'].to_frame('SPY')
df_spy_close.to_csv('./data/df_spy_close.csv')

[*********************100%***********************]  4 of 4 completed
[*********************100%***********************]  1 of 1 completed


## Percent Change for recent past year

In [129]:
recent_pct_change = df_recent_close.pct_change().iloc[1:] #get rid of last row
recent_mean_return = recent_pct_change.mean() * 365
recent_risk = recent_pct_change.var() * 365 
recent_cov_mat = recent_pct_change.cov() * 365

recent_risk

COST    0.142896
DIS     0.199490
MSFT    0.181635
NKE     0.259891
dtype: float64

In [130]:
plot_line(recent_pct_change, 'Date', 'Percent', 'Percent Change (daily return)')

In [131]:
spy_pct_change = df_spy_close.pct_change().iloc[1:] #get rid of last row
spy_mean_return = spy_pct_change.mean() * 365
spy_risk = spy_pct_change.var() * 365

In [132]:
plot_line(spy_pct_change, 'Date', 'Percent', 'Percent Change (daily return)')

## Cumulative Product Past Recent Year

In [133]:
recent_cumulative_product = recent_pct_change.add(1).cumprod().sub(1)
recent_cumulative_product.tail()

,COST,DIS,MSFT,NKE
Date,,,,
2022-12-23 00:00:00-05:00,-0.183621,-0.438568,-0.286841,-0.294043
2022-12-27 00:00:00-05:00,-0.190944,-0.449030,-0.292128,-0.286087
2022-12-28 00:00:00-05:00,-0.200667,-0.463065,-0.299388,-0.301755
2022-12-29 00:00:00-05:00,-0.194420,-0.443863,-0.280030,-0.287363
2022-12-30 00:00:00-05:00,-0.194473,-0.445777,-0.283585,-0.289427


In [134]:
plot_line(recent_cumulative_product, 'date', 'cumulative product', 'cumulative product' )

In [135]:
spy_cumulative_product = spy_pct_change.add(1).cumprod().sub(1)
spy_cumulative_product.tail()

,SPY
Date,
2022-12-23 00:00:00-05:00,-0.198447
2022-12-27 00:00:00-05:00,-0.201608
2022-12-28 00:00:00-05:00,-0.211530
2022-12-29 00:00:00-05:00,-0.197337
2022-12-30 00:00:00-05:00,-0.199452


In [136]:
plot_line(spy_cumulative_product, 'date', 'cumulative product', 'cumulative product')

In [137]:
recent_return = max_sharpe['Weights'] @ (recent_mean_return)
recent_return

-0.3355461590031889

In [138]:
spy_recent_return = spy_mean_return['SPY']
spy_recent_return

-0.2819999334836532

In [139]:
recent_risk = ((max_sharpe['Weights'] @ recent_cov_mat) @ max_sharpe['Weights'])**(1/2)
recent_risk

0.36606734561980064

In [140]:
spy_risk = spy_risk['SPY']
spy_risk

0.08558675698623609